<font size=7>  $\textrm{Tutorial} \ 2$  </font>

<font size=5>  $\textrm{Supervised learning phase separation}$   </font>



<font size=4>  $\textrm{We will try to learn the classification of phases into ferromagnetic/paramagnetic based on spin snapshots.}$   </font>

<font size=4>  $\textrm{Our goal: Train a machine learning model that will take in a spin configuration and tell us which phase it lives in for the Ising Hamiltonian.}$   </font>


<font size=5> $H = -J \sum_{\langle i, j \rangle}^{}  \sigma_i^z \sigma_{j}^z $</font>

<font size=5>  $\textrm{a) Preparing the data}$   </font>

<font size=4>  $\textrm{For this we will use the following function, that uses the Metropolis Monte Carlo algorithm to generate the samples at a fixed temperature.}$   </font>

In [136]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

In [2]:
L = 10 # number of lattice sites L x L
N_spins = L**2

In [137]:
### Store each spin's four nearest neighbours in a neighbours array (using periodic boundary conditions):
neighbours = np.zeros((N_spins,4), dtype=np.int64)
for i in range(N_spins):
    #neighbour to the right:
    neighbours[i,0]=i+1
    if i%L==(L-1):
        neighbours[i,0]=i+1-L
  
    #upwards neighbour:
    neighbours[i,1]=i+L
    if i >= (N_spins-L):
        neighbours[i,1]=i+L-N_spins

    #left neighbour:
    neighbours[i,2] = i-1
    if i%L==0:
        neighbours[i,2]=i-1+L

    #downwards neighbour:
    neighbours[i,3] = i-L
    if i < L:
        neighbours[i,3]= N_spins-L+i




In [140]:
def calcEnergy(config):
    '''Energy of a given configuration'''
    energy = 0
    for i in range(len(config)):
        for j in range(len(config)):
            S = config[i,j]
            nb = config[(i+1)%N, j] + config[i,(j+1)%N] + config[(i-1)%N, j] + config[i,(j-1)%N]
            energy += -nb*S
    return energy/4.

In [5]:
def metropolis_step(spins, beta, J=1):
    """
    Perform one step of the Metropolis algorithm.
    
    Parameters:
        spins (np.array): Current configuration of spins.
        T (float): Temperature of the system.
        J (float): Interaction energy coefficient.
        
    Returns:
        np.array: Updated configuration of spins.
    """
    N = len(spins)
    i = np.random.randint(N)  # Randomly pick a spin to flip
    spins[i] *= -1            # Flip the spin
    

    delta_E = -2 * J * spins[i] * (spins[neighbours[i,0]] + spins[neighbours[i,1]] + spins[neighbours[i,2]] + spins[neighbours[i,3]])


    # Metropolis criterion
    if delta_E < 0 or np.random.rand() < np.exp(-1*delta_E * beta):
        # Accept the flip
        return spins
    else:
        # Revert the flip (reject)
        spins[i] *= -1
        return spins


In [6]:
def run_metropolis(N, beta,numSamples,J=1, sample_interval=10):
    """
    Run the Metropolis Monte Carlo algorithm to sample spin configurations after allowing the system to equilibrate,
    and with a specified interval between samples to reduce autocorrelation.
    
    Parameters:
        N (int): Lattice size
        T (float): Temperature.
        J (float): Interaction energy coefficient.
        steps (int): Total number of Metropolis steps to perform.
        equilibration_steps (int): Number of steps to discard at the start to allow the system to equilibrate.
        sample_interval (int): Interval of steps between collecting samples to reduce autocorrelation.
        
    Returns:
        list: A list of sampled spin configurations.
    """
    
    equilibration_steps=20*N**2
    steps=equilibration_steps + sample_interval*numSamples
    
    spins = np.random.choice([-1, 1], size=N**2)  # Initial random spins
    samples = []
    
    for step in range(steps):
        spins = metropolis_step(spins, beta, J)  
        
        # Only start collecting samples after the equilibration period
        if step >= equilibration_steps and (step - equilibration_steps) % sample_interval == 0:
            samples.append(spins.copy())  # Collect samples at specified intervals

        if step%200 == 0:
            spins *= -1 #Global spin flip to avoid studying just 1 basin
    
    return np.array(samples)


In [7]:
# Generate the ferromagnetic phase data
beta = 1/(1.5) # Inverse Temperature
number_samples = 5000
samples_ferro = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [8]:
label_ferro = np.zeros(5000)

In [9]:
# Generate the paramagnetic phase data
beta = 1/(5) # Inverse Temperature
number_samples = 5000
samples_para = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [10]:
label_para = np.ones(5000)

<font size=4>  $\textrm{We will also go ahead and generate two smaller sets of 2500 each as a test data set. These won't be used for training but rather to validate if our model is learning correctly.}$   </font>

In [13]:
# Generate the ferromagnetic phase data
beta = 1/(1.5) # Inverse Temperature
number_samples = 2500
samples_ferro_test = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [14]:
label_ferro_test = np.zeros(2500)

In [15]:
# Generate the paramagnetic phase data
beta = 1/(5) # Inverse Temperature
number_samples = 2500
samples_para_test = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [135]:
label_para_test = np.ones(2500)

<font size=5>  $\textrm{b) Training the model:}$   </font>

<font size=4>  $\textrm{Setting up and organizing our data. Now we will organize our data such that Pytorch will recognize it.}$   </font>

In [54]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [18]:
class SpinConfigurationDataset(Dataset):
    def __init__(self, spin_configs, labels):
        self.spin_configs = torch.tensor(spin_configs, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        spin_config = self.spin_configs[idx]
        label = self.labels[idx]
        return spin_config, label

In [19]:
training_data = torch.tensor(np.concatenate((samples_ferro,samples_para)))

In [20]:
training_labels = torch.tensor(np.concatenate((label_ferro,label_para )))

In [21]:
dataset = SpinConfigurationDataset(training_data, training_labels)

/tmp/ipykernel_3722274/564826694.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.spin_configs = torch.tensor(spin_configs, dtype=torch.float32)
/tmp/ipykernel_3722274/564826694.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


In [22]:
# Create data loader
batch_size = 500
shuffle = True
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [23]:
test_data = torch.tensor(np.concatenate((samples_ferro_test,samples_para_test)))

In [24]:
test_labels = torch.tensor(np.concatenate((label_ferro_test,label_para_test )))

In [25]:
test_data_loader = SpinConfigurationDataset(test_data, test_labels)

/tmp/ipykernel_3722274/564826694.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.spin_configs = torch.tensor(spin_configs, dtype=torch.float32)
/tmp/ipykernel_3722274/564826694.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


In [26]:
# Create data loader
batch_size = 500
shuffle = True
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [27]:
data_loader

In [33]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

# Define the neural network architecture
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        # x = self.sigmoid(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)

In [34]:
# Define hyperparameters
input_size = L*L 
hidden_size = 100
output_size = 2  # Number of classes
learning_rate = 0.001
num_epochs = 30

The `nn.CrossEntropyLoss` in PyTorch combines the operations of `nn.LogSoftmax` and `nn.NLLLoss` (negative log likelihood loss) in a single class. This is a commonly used loss function for classification tasks, particularly for multi-class classification (although ours is binary, but this work more generally for you to try stuff on your own!).

Here's a detailed explanation of how `nn.CrossEntropyLoss` works:

### Step-by-Step Breakdown

1. **Raw Model Outputs (Logits):**
   - Your neural network outputs raw scores, called logits, for each class. These logits are not probabilities and can take any value (positive or negative).
   - Let's denote the logits for a single sample as $\mathbf{z} = [z_1, z_2, \ldots, z_C]$, where $C$ is the number of classes.

2. **Softmax Function:**
   - The softmax function is applied to the logits to convert them into probabilities. The softmax function is defined as:
     <font size=4>$
     \sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^C e^{z_j}}
     $</font>
   - This ensures that the output probabilities are in the range $[0, 1]$ and sum to $1$.

3. **Log-Softmax:**
   - Instead of applying the softmax function and then taking the logarithm, `nn.CrossEntropyLoss` directly computes the log-softmax. This is more numerically stable. The log-softmax for class \(i\) is defined as: 
     <font size=4>$
     \ \log(\sigma(\mathbf{z})_i) = z_i - \log\left(\sum_{j=1}^C e^{z_j}\right)
     $</font>

4. **Negative Log Likelihood Loss (NLLLoss):**
   - The loss for a single sample with true class label $y$ is given by:
     $
     \ \text{NLLLoss} = -\log(\sigma(\mathbf{z})_y)
     $
   - This corresponds to the negative log probability of the true class.

5. **Combining the Steps:**
   - `nn.CrossEntropyLoss` combines the log-softmax and NLLLoss steps into one function for efficiency. The final loss for a batch of samples is the average (or sum, depending on the settings) of the individual losses.

### Formula for Cross Entropy Loss
The cross-entropy loss for a single sample can be written as:
<font size=4>$\text{CrossEntropyLoss}(\mathbf{z}, y) = - \log\left(\frac{e^{z_y}}{\sum_{j=1}^C e^{z_j}}\right)$ </font>
where $z_y$ is the logit corresponding to the true class $y$.

For a batch of $N$ samples, the average cross-entropy loss is:
<font size=4>$
\text{CrossEntropyLoss}(\mathbf{Z}, \mathbf{y}) = -\frac{1}{N} \sum_{i=1}^N \log\left(\frac{e^{z_{i,y_i}}}{\sum_{j=1}^C e^{z_{i,j}}}\right)
$</font>
where $z_{i,y_i}$ is the logit for the $i$-th sample corresponding to its true class $y_i$.

### Practical Example in PyTorch

Here's how we typically implement it `nn.CrossEntropyLoss` in a PyTorch model:

In [143]:
# Instantiate the model
model = FFNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for inputs, targets in data_loader:
        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()


    # Print average loss for this epoch
    # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data_loader)}")
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data_loader)}, Test Loss: {test_loss / len(test_data_loader)}")

Epoch [1/30], Loss: 0.44697161167860033, Test Loss: 0.29559473545849324
Epoch [2/30], Loss: 0.21996216028928756, Test Loss: 0.1585379368290305
Epoch [3/30], Loss: 0.11760344505310058, Test Loss: 0.08671096526607872
Epoch [4/30], Loss: 0.06500279381871224, Test Loss: 0.05117717612423003
Epoch [5/30], Loss: 0.03962917514145374, Test Loss: 0.0332772844074294
Epoch [6/30], Loss: 0.026622920669615267, Test Loss: 0.02378743175319396
Epoch [7/30], Loss: 0.019471199670806526, Test Loss: 0.018109327238239346
Epoch [8/30], Loss: 0.015066803805530072, Test Loss: 0.014418980653700418
Epoch [9/30], Loss: 0.012151133362203836, Test Loss: 0.011851019423455
Epoch [10/30], Loss: 0.010100263077765704, Test Loss: 0.01004483562564128
Epoch [11/30], Loss: 0.008610045164823532, Test Loss: 0.008695519871870056
Epoch [12/30], Loss: 0.0074588393326848745, Test Loss: 0.007635266881051939
Epoch [13/30], Loss: 0.0065645945724099874, Test Loss: 0.006802698300644988
Epoch [14/30], Loss: 0.005844850826542824, Test L

In [157]:
inputs

tensor([ 1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,
        -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1.,
         1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
         1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
        -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,
        -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
        -1.,  1.])

In [154]:
F.softmax(model(inputs))

/tmp/ipykernel_3722274/4102808501.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(model(inputs))


tensor([3.4194e-04, 9.9966e-01], grad_fn=<SoftmaxBackward0>)

<font size=5>  $\textrm{c) Testing generalization:}$   </font>

In [37]:
# Generate the ferromagnetic phase data
beta = 1/(0.3) # Inverse Temperature
number_samples = 2500
General_ferro_test = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [38]:
label_ferro_general = np.zeros(2500)

In [39]:
# Generate the paramagnetic phase data
beta = 1/(3.5) # Inverse Temperature
number_samples = 2500
General_para_test = run_metropolis(L, beta,number_samples,sample_interval=10*N_spins)

In [80]:
label_para_general = np.ones(2500)

In [81]:
gene_data = torch.tensor(np.concatenate((General_ferro_test,General_para_test)))

gene_labels = torch.tensor(np.concatenate((label_ferro_general,label_para_general)))

In [82]:
dataset_gen = SpinConfigurationDataset(gene_data, gene_labels)

/tmp/ipykernel_3722274/564826694.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.spin_configs = torch.tensor(spin_configs, dtype=torch.float32)
/tmp/ipykernel_3722274/564826694.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


In [83]:
# Create data loader
batch_size = 500
shuffle = True
data_loader_gene = DataLoader(dataset_gen, batch_size=batch_size, shuffle=shuffle)

In [159]:
# Validation
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in data_loader_gene:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

In [160]:
print(f"Test Loss: {test_loss / len(test_data_loader)}")

Test Loss: 1.776701295748353e-05


<font size=5>  $\text{Benchmarking our model}$   </font>

<font size=4> $\text{Precision}$ </font>
\begin{equation}
\text{Precision} = \frac{TP}{TP + FP}
\end{equation}

<font size=4> $\text{Recall (Sensitivity or True Positive Rate)}$ </font>
\begin{equation}
\text{Recall} = \frac{TP}{TP + FN}
\end{equation}

<font size=4> $\text{F1 Score}$ </font>
\begin{equation}
\text{F1 Score} = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
\end{equation}

<font size=4> $\text{Accuracy}$ </font>
\begin{equation}
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
\end{equation}

<font size=4> $\text{False Positive Rate}$ </font>
\begin{equation}
\text{FPR} = \frac{FP}{FP + TN}
\end{equation}



In [134]:
# Accuracy:  The ratio of correctly predicted samples to the total number of samples.

# Precision: The ratio of true positive predictions to the total number of positive predictions (true positives + false positives). 

# Recall: The ratio of true positive predictions to the total number of actual positives (true positives + false negatives).

# F1 Score: The harmonic mean of precision and recall. It balances both metrics.

In [161]:
# Set the model to evaluation mode
model.eval()

# Perform a forward pass to get the logits
with torch.no_grad():
    logits = model(inputs)

In [162]:
# Apply sigmoid to get probabilities
probs = F.softmax(logits,dim=1)


# Determine the predicted class (0 or 1) based on the threshold of 0.5
predictions = (probs >= 0.5).long()

In [165]:
# Convert predictions and labels to numpy arrays for scikit-learn functions
predictions_np = np.argmax(predictions.cpu().numpy(), axis=1)
labels_np = targets.cpu().numpy()

In [166]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [167]:
labels_np

array([1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [168]:
# Compute accuracy
accuracy = accuracy_score(labels_np, predictions_np)

# Compute precision
precision = precision_score(labels_np, predictions_np)

# Compute recall
recall = recall_score(labels_np, predictions_np)

# Compute F1 score
f1 = f1_score(labels_np, predictions_np)



# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.9980
Precision: 1.0000
Recall: 0.9957
F1 Score: 0.9979
